In [1]:
import requests as rq
import pandas as pd
import re

In [2]:
apikey = "it's a secret"

In [3]:
payload = {}
headers = {'apikey': apikey}

base_url = "https://api.apilayer.com/spotify/"
endpoint_url = "album_tracks"
album_url = "album_metadata"

In [4]:
songs = pd.DataFrame(columns=['Title','Album','Track Number','Play Count','Duration in Seconds','Rating'])

In [5]:
ids = ['7mzrIsaAjnXihW3InKjlC3','7vzYp7FrKnTRoktBYsx9SF','2dqn5yOQWdyGwOpOIi9O4x','4hDok0OAJd57SGIT8xuWJH',
       '5EpMjweRD573ASl7uNiHym','1KlU96Hw9nlvqpBPlSqcTV','6kZ42qRrzov54LcAk4onW9','34OkZVpuzBa9y40DCy0LPR',
       '6DEjYFkNZh67HP7R9PSZvv','1NAmidJlEaVgA3MpcPFYGq','1pzvBxYgT6OVwJLtHkrdQK','6AORtDjduMM3bupSWzbTSG',
       '3lS1y25WAhcqJDATJK70Mq']

In [6]:
for j in range(len(ids)):
    params = {'id': ids[j]}
    r = rq.get(base_url + endpoint_url, params=params, headers=headers, data=payload)
    data = pd.DataFrame(r.json())
    data = data["data"].apply(pd.Series)
    data = data["tracks"].apply(pd.Series)
    data = data["items"].apply(pd.Series)
    table = pd.DataFrame(columns=['Title','Album','Track Number','Play Count','Duration in Seconds','Rating'])
    for i in range(data.shape[1]):
        track = data.iloc[0,i]
        table.loc[i,'Title'] = track['track']['name']
        table.loc[i,'Track Number'] = track['track']['trackNumber']
        table.loc[i,'Play Count'] = track['track']['playcount']
        table.loc[i,'Duration in Seconds'] = track['track']['duration']['totalMilliseconds']/1000
        table.loc[i,'Rating'] = track['track']['contentRating']['label']
    ra = rq.get(base_url + album_url, params=params, headers=headers, data=payload)
    data = pd.DataFrame(ra.json())
    table['Album'] = data['data']['album']['name']
    songs = pd.concat([songs,table])

In [7]:
sids = ['45nqVXRAW0xv0wpU9JljPN','1O3BsjGx9plSOJ036ZY4Fl','1MPAXuTVL2Ej5x0JHiSPq8', '5JnnrTE35MQagEnEkFthiF']
num = [13, 10, 9, 0]

In [8]:
for j in range(len(sids)):
    params = {'id': sids[j]}
    r = rq.get(base_url + endpoint_url, params=params, headers=headers, data=payload)
    data = pd.DataFrame(r.json())
    data = data["data"].apply(pd.Series)
    data = data["tracks"].apply(pd.Series)
    data = data["items"].apply(pd.Series)
    table = pd.DataFrame(columns=['Title','Album','Track Number','Play Count','Duration in Seconds','Rating'])
    track = data.iloc[0,num[j]]
    table.loc[i,'Title'] = track['track']['name']
    table.loc[i,'Track Number'] = 1
    table.loc[i,'Play Count'] = track['track']['playcount']
    table.loc[i,'Duration in Seconds'] = track['track']['duration']['totalMilliseconds']/1000
    table.loc[i,'Rating'] = track['track']['contentRating']['label']
    table['Album'] = 'Single'
    songs = pd.concat([songs,table])

In [9]:
songs['Track Number']=songs['Track Number'].apply(lambda x: pd.to_numeric(x))
songs['Play Count']=songs['Play Count'].apply(lambda x: pd.to_numeric(x))
songs['Duration in Seconds']=songs['Duration in Seconds'].apply(lambda x: pd.to_numeric(x))

In [10]:
songs['Album'] = songs['Album'].str.split(r'\ \(',expand=True)[0]
songs['Title'] = songs['Title'].str.split(r'\ \(',expand=True)[0]
songs['Title'] = songs['Title'].str.split(r'\ -',expand=True)[0]
songs['Album'] = songs['Album'].str.strip()
songs['Title'] = songs['Title'].str.strip()

In [11]:
songs_final = songs.drop_duplicates(subset=['Title'], keep='first')
songs_final = songs_final.drop(['Play Count','Duration in Seconds'], axis=1)
songs_final['Play Count'] = list(songs.groupby([songs['Title']]).sum()['Play Count'])
songs_final['Duration in Seconds'] = list(songs.groupby([songs['Title']]).mean()['Duration in Seconds'])
songs_final.reset_index(inplace=True)
songs_final = songs_final.drop('index', axis=1)

In [12]:
songs_final

,Title,Album,Track Number,Rating,Play Count,Duration in Seconds
0,Tim McGraw,Taylor Swift,1,NONE,448337012,208.1860
1,Picture To Burn,Taylor Swift,2,NONE,421937510,230.5465
2,Teardrops On My Guitar,Taylor Swift,3,NONE,14840936,220.1460
3,A Place in this World,Taylor Swift,4,NONE,24269863,199.2000
4,Cold As You,Taylor Swift,5,NONE,201250577,223.2930
...,...,...,...,...,...,...
195,Dear Reader,Midnights,20,NONE,214722354,231.0000
196,Eyes Open,Single,1,NONE,212032914,195.1060
197,Highway Don't Care,Single,1,NONE,147470813,245.4400
198,I Don’t Wanna Live Forever,Single,1,NONE,543239713,214.7060


In [13]:
songs_final.to_csv('TaylorSwiftSongs.csv')